## Import stuff

In [3]:
import numpy as np
import copy

# Import plotting stuff
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib import cm
from matplotlib.patches import Rectangle

# Import skimage and scipy stuff
from skimage.morphology import remove_small_objects, binary_erosion, binary_dilation, disk, erosion, dilation, opening, closing, medial_axis, skeletonize
from skimage.color import rgb2grey
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.measure import regionprops, label
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import peak_local_max
from skimage.filters import gaussian
from skimage import measure
from skimage.graph import route_through_array
from skimage.io import imread
from scipy import ndimage
from scipy import sparse

#Import skan stuff
from numba.experimental import jitclass
from skan import Skeleton, summarize
from skan import skeleton_to_csgraph
from skan import draw

scale=None
%matplotlib qt

def addScaleBar(ax):
    """Add a scale bar to an axes.
    """
    if scale:
        scalebar = ScaleBar(scale)
        fig.gca().add_artist(scalebar)
        plt.show()

def addArrows(ax, c='r', lenx=0.04, leny=0.06, flip=False):
    """Add coordinate definition arrows (radial and circumferential) to an axes.
    """
    startcoord = (0.02, 0.02);
    
    ax.annotate("", xy=(startcoord[0]-0.002, startcoord[1]), xytext=(startcoord[0]+lenx, startcoord[1]), xycoords = 'axes fraction', c=c, arrowprops=dict(arrowstyle="<-", color=c, lw=3))
    ax.annotate("", xy=(startcoord[0], startcoord[1]-0.002), xytext=(startcoord[0], startcoord[1]+leny), xycoords = 'axes fraction', c=c, arrowprops=dict(arrowstyle="<-", color=c, lw=3))
    if flip == False:
        ax.annotate("R", xy=(0.011, startcoord[1]+leny), xycoords = 'axes fraction', fontsize=18, fontweight='bold', c=c)
        ax.annotate("C", xy=(startcoord[0]+lenx, 0.01), xycoords = 'axes fraction', fontsize=18, fontweight='bold', c=c)
    if flip == True:
        ax.annotate("C", xy=(0.011, startcoord[1]+leny), xycoords = 'axes fraction', fontsize=18, fontweight='bold', c=c)
        ax.annotate("R", xy=(startcoord[0]+lenx, 0.01), xycoords = 'axes fraction', fontsize=18, fontweight='bold', c=c)

# Run cropImage(im) to crop an image
def cropImage(im, crop_bottom, crop_top, crop_left, crop_right):
    """Crop an image.
    """
    if crop_bottom == 0:
        im = im[crop_top:]
    else:
        im = im[crop_top:-crop_bottom]
    if crop_right == 0:
        im = im[:, crop_left:]
    else:
        im = im[:, crop_left:-crop_right]
    return im

def plot_comparison(original, filtered, orig_name, filter_name):
    """Plotting two plots next to each other.
    """
    fig, (ax_a, ax_b) = plt.subplots(ncols=2, figsize=(26, 16), sharex=True, sharey=True)
    ax_a.imshow(original, cmap=plt.cm.gray)
    ax_a.set_title(orig_name, fontsize=16)
    ax_a.axis('off')
    ax_b.imshow(filtered, cmap=plt.cm.gray)
    ax_b.set_title(filter_name, fontsize=16)
    ax_b.axis('off')
    addArrows(ax_a)
    addArrows(ax_b)
    addScaleBar(ax_b)
    
def nan_gaussian(arr, sigma):
    """Apply a gaussian filter to an array with nans.
    """
    nan_msk = np.isnan(arr)
    loss = np.zeros(arr.shape)
    loss[nan_msk] = 1
    loss = ndimage.gaussian_filter(loss, sigma=sigma, mode='constant', cval=1)
    gauss = arr.copy()
    gauss[nan_msk] = 0
    gauss = ndimage.gaussian_filter(gauss, sigma=sigma, mode='constant', cval=0)
    gauss[nan_msk] = np.nan
    gauss += loss * arr

    return gauss

## Import image

Enter filename after `image=` - will require full path unless it's in the same folder as the notebook. It is also possible to crop the image here `crop_bottom`, `crop_top`, `crop_left` and `crop_right`. Radial is defined as vertical - if necessary, set `transpose` to `True` to make radial along vertical. Set `scale` to the size of a pixel in metres:

In [4]:
###################################################################
image_path = 'colas5.jpg'
crop_bottom = 0; crop_top = 0; crop_left = 0; crop_right = 0;
transpose = False
scale = 0.08e-3/212
###################################################################

original_image = imread(image_path)
original_image = rgb2grey(original_image)
if transpose == True:
    original_image = np.transpose(original_image)
original_image = cropImage(original_image, crop_bottom, crop_top, crop_left, crop_right)

print('Loaded image: {0}'.format(image_path))
print('Dimensions: {0} x {1} px'.format(original_image.shape[1], original_image.shape[0]))

if scale is not None:
    scalebar = ScaleBar(scale)
    print('\nOne pixel = {0:.3f} um'.format(scale*1e6))
    print('Dimensions: {0:.1f} x {1:.1f} mm'.format(original_image.shape[1]*scale*1e3, original_image.shape[0]*scale*1e3))

fig, ax = plt.subplots(figsize=(16,16))
plt.axis('off')
plt.imshow(original_image, cmap='gray')
plt.title('Original image', fontsize=16)
addArrows(ax)
addScaleBar(ax)

fig, ax = plt.subplots(figsize=(10,4))
ax.hist(original_image.flatten(), bins=80, range=(0,1));
ax.set_title('Image Histogram', fontsize=16);
ax.set_xlabel('Gray value', fontsize=14); ax.set_ylabel('Frequency', fontsize=14);

Loaded image: colas5.jpg
Dimensions: 437 x 437 px

One pixel = 0.377 um
Dimensions: 0.2 x 0.2 mm


## Crop black edges of image

Find large darks block (like at the edges of images) and crop these - check that hydrides are not cropped.
Change `crop_param`, `size_param` and `dilation_param` if necessary.

In [5]:
##############################################################################################
# Threshold used for removing dark edges, use the histogram above to determine this value
crop_param = 0.0; 

# Make sure features below this size (i.e. hydrides) are not included in cropping
size_param = 500

# Dilate the cropped boundary by this many pixels
dilation_param = 5
##############################################################################################

crop_threshold = original_image < crop_param
crop_threshold = remove_small_objects(crop_threshold, size_param)

crop_threshold = binary_dilation(crop_threshold, selem=disk(dilation_param))        
cropped_image = copy.deepcopy(original_image)
cropped_image[crop_threshold] = np.nan

plot_comparison(crop_threshold, cropped_image, 'Cropping Threshold (white is cropped)', 'Cropped Image')

fig, ax = plt.subplots(figsize=(10,6))
ax.hist(cropped_image[~np.isnan(cropped_image)].flatten(), bins=80, range=[0,1]);
ax.set_title('Image Histogram', fontsize=16);
ax.set_xlabel('Gray value', fontsize=14); ax.set_ylabel('Frequency', fontsize=14);

## Try to minimise grain contrast

Divide the image by a Gaussian blur of the same image to try to remove grain contrast and variations in brightness across the image. Change `sigma` to change the Gaussian blur radius

In [6]:
##################
sigma = 50
##################

fig, ax1 = plt.subplots(figsize=(14,8))
plt.axis('off')
gaussian_blur = nan_gaussian(cropped_image, sigma = sigma)
removedGrains = cropped_image/gaussian_blur
ax1.imshow(removedGrains, cmap='gray')
ax1.set_title('Remove local contrast variation', fontsize=16);
addArrows(ax1)
addScaleBar(ax1)

fig, ax = plt.subplots(figsize=(10,6))
ax.hist(removedGrains[~np.isnan(removedGrains)].flatten(), bins=60, range=(0,2));
ax.set_title('Image Histogram', fontsize=16);
ax.set_xlabel('Gray value', fontsize=14); ax.set_ylabel('Frequency', fontsize=14);

## Simple threshold

Change the threshold value by changing the `threshold` variable and remove small objects (noise), below a certain size defined by the `small_object` paramater. Use the original image histogram to determine the thresholding limit

In [7]:
####################################
threshold = 0.90
small_object = 250
####################################

thres = removedGrains < threshold
thres = remove_small_objects(thres, min_size=small_object)

# cropped = nan, hydride = 1, no hydride = 0
thres_disp = copy.deepcopy(thres)
thres_disp = np.array(thres_disp)*1.0
thres_disp[crop_threshold] = np.nan
           
cropped_image = copy.deepcopy(removedGrains)
cropped_image[crop_threshold] = np.nan

plot_comparison(thres_disp, cropped_image,'Thresholded Image', 'Cropped Image')

fig, ax = plt.subplots(figsize=(10,6))
ax.hist([cropped_image[thres_disp==1].flatten(), cropped_image[thres_disp==0].flatten()], stacked=True, label=['Hydride', 'Background'], bins=60, range=(0,2));
ax.set_title('Image Histogram', fontsize=16);
ax.set_xlabel('Gray value', fontsize=14); ax.set_ylabel('Frequency', fontsize=14);
plt.legend();

## Hough Line Transform For Hydrides

Here there are three parameters that you can change that are input into the Hough Transform:
`num_peaks` = max number of lines drawn in each bounding box
`min_distance` = minimum distance separating two parallel lines
`min_angle` = minimum angle separating two lines

Here, bounding boxes are drawn around each hydride and the hough line tranform is calculated for each hydride. 
Importantly, the threshold is set as `threshold = 0.25*np.amax(h)` this means that only hydrides that are at least 0.25 times the length of the longest hydride are measured. This can also be changed dependednt of needs, but increases the noise associated with the hough transform. 

In [25]:
####################################
# Maximum number of lines
num_peaks=20
# Minimum hough space distance between each detected line should be less than 10 pixels
min_distance=5
# Minimum hough space angle between each detected line #10 degrees too large
min_angle=4
####################################
fig, axes = plt.subplots(2, 1, figsize=(30,35))
ax = axes.ravel()
# Read in image (if you erode the image read in that image here instead of thres)
image2 = thres
# Plotting
ax[0].imshow(image2, cmap='gray')
ax[0].set_axis_off()
ax[0].set_title('Thresholded image', fontsize=16)
addArrows(axes[0])
# Label image
label, num_features = ndimage.label(image2 > 0.1)
slices = ndimage.find_objects(label)
# Loop over each slice
len_list = np.zeros([0]); angle_list = np.zeros([0]); d_list = np.zeros([0]);
for feature in np.arange(num_features):
    h, theta, d = hough_line(label[slices[feature]], theta=np.linspace(-np.pi/2 , np.pi/2 , 90))
    threshold = 0.25*np.amax(h)
    h_peak, angles, d_peak = hough_line_peaks(h, theta, d, threshold=threshold, num_peaks=num_peaks, min_distance=min_distance, min_angle=min_angle)
    angle_list = np.append(angle_list,angles)
    len_list = np.append(len_list,h_peak)
    d_list = np.append(d_list, d_peak)
    # Draw bounding box
    x0_box = np.min([slices[feature][1].stop, slices[feature][1].start])
    y0_box = np.min([slices[feature][0].stop, slices[feature][0].start])
    x1_box = np.max([slices[feature][1].stop, slices[feature][1].start])
    y1_box = np.max([slices[feature][0].stop, slices[feature][0].start])
    rect = Rectangle((x0_box, y0_box), x1_box-x0_box, y1_box-y0_box, angle=0.0, ec='r', fill=False)
    ax[0].add_artist(rect)
    origin=np.array((0, np.abs(x1_box-x0_box)))
    for _, angle, dist in zip(h_peak, angles, d_peak):
        y0b, y1b = (dist - np.array((0, x1_box-x0_box)) * np.cos(angle)) / np.sin(angle)
        y0_line = y0b+y0_box; y1_line = y1b+y0_box
        x0_line = x0_box; x1_line = x1_box
        m = (y1_line-y0_line)/(x1_line-x0_line)
        # Fix lines which go over the edges of bounding boxes
        if y0_line < y0_box: x0_line = ((y0_box - y1_line) / m) + x1_line; y0_line = y0_box;
        if y0_line > y1_box: x0_line = ((y1_box - y1_line) / m) + x1_line; y0_line = y1_box;
        if y1_line < y0_box: x1_line = ((y0_box - y1_line) / m) + x1_line; y1_line = y0_box;
        if y1_line > y1_box: x1_line = ((y1_box - y1_line) / m) + x1_line; y1_line = y1_box;
        ax[0].plot(np.array((x0_line, x1_line)), (y0_line, y1_line), '-g')
print('Number of detected angles: {0}'.format(len(len_list)))
ax[0].set_xlim(0,image2.shape[1]); ax[0].set_ylim(0,image2.shape[0]);
# Radial/circumferential fractions
radial = np.sum(np.where(np.logical_and(-np.pi / 4 <= angle_list, angle_list < np.pi / 4), len_list, 0))
circumferential = np.sum(np.where(np.logical_and(-np.pi / 4 <= angle_list, angle_list < np.pi / 4), 0, len_list))
print('Radial Hydride Fraction: {0:.3f} \n Circumferential Hydride Fraction: {1:.3f}'.format(radial/(radial+circumferential),circumferential/(radial+circumferential))   )
ax[1].scatter(np.rad2deg(angle_list),len_list,color='r')
ax[1].set_xlabel('Angle ($\degree$)', fontsize=14); ax[1].set_ylabel('Hough space peak intensity', fontsize=14);
ax[1].set_xlim(-90, 90)
plt.savefig('plot.pdf')
plt.show()

Number of detected angles: 363
Radial Hydride Fraction: 0.317 
 Circumferential Hydride Fraction: 0.683


## Radial Hydride Fraction Determination With Projection Factor

This value is calculated based on the RHF proposed by Colas et al (add ref)
Note that the angles produced by the hough line transform are reversed ( an angle of 0 = fully radial and an angle of 90 = fully circumferential)

In [26]:
#convert radian values to degrees
deg_angle_list = np.rad2deg(angle_list)
fi = []

for k in deg_angle_list:
    if k >0 and k<=30: x=1
    elif k>30 and k<=50: x=0.5
    elif k>50 and k<=90: x=0
    elif k>-30 and k<=0: x=1
    elif k>-50 and k<=-30: x=0.5
    elif k>-90 and k<=-50: x=0
        
    fi.append(x)

#The next step is to do the summation
SumOfLixFi = sum(len_list*np.array(fi))
SumOfLi = sum(len_list)

print('The total length of hydrides is: {0:.1f} um'.format(SumOfLi))

RHF = SumOfLixFi/SumOfLi
print('The radial hydride fraction is:  {0:.3f}'.format(RHF))

The total length of hydrides is: 11459.0 um
The radial hydride fraction is:  0.372


## Connectivity factor

Here we plot the path of least resistance from the top to the bottom of the microstrucutre. 
We have added a scaling function to make moving in the horizonal direction more costly --> this is because in reality hydrides oriented in the circumferential direction do not cause rapid crack propagation and failure. 

In [27]:
con_img = thres

edist = ndimage.morphology.distance_transform_edt(con_img==0)
edist = rescale(edist,(1,1))

#add a row of zeros on the top and bottom 
edist[0,:]=0
edist[-1,:]=0

path, cost = route_through_array(edist,[0,0],[-1,-1])
path2=np.array(path)
path3 = []
for item in path2:
    if item[0] != 0:
        if item[0]!= edist.shape[0]-1:
            path3.append(item)

path4 = np.array(path3)

plt.imshow(edist)
plt.colorbar()
plt.scatter(path4[:,1], path4[:,0], s=3, c='r')

## Branching Length Fraction

In [8]:
branched = thres
skeleton = skeletonize(branched)
plt.imshow(skeleton, cmap ='gray')
#plt.show()


#Label all of the unique hydrides
label, num_features = ndimage.label(skeleton,structure = [[1,1,1],[1,1,1],[1,1,1]])

total_length = np.sum(skeleton)
total_branch_length = 0
for l in range(num_features):
    feature_image = label == l + 1
    pixel_graph, coordinates, degrees = skeleton_to_csgraph(label == l+1)
    shortest_distance = sparse.csgraph.shortest_path(pixel_graph)
    shortest_distance= np.where(shortest_distance == np.inf, 0, shortest_distance)
    main_hydride_length = np.amax(shortest_distance)
    total_branch_length += np.sum(feature_image) - main_hydride_length

#Ask how to plot a pretty image     
BLF =  total_branch_length/total_length    
print('The Branch Length Fraction = ',BLF)


The Branch Length Fraction =  0.25383610680259533
